In [1]:
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
import os

# Make Sampling

In [2]:
negative_sample_ratio=0.9

In [3]:
from_path = '../criteo_x1'
to_path = '../criteo_x1_{}'.format(negative_sample_ratio)
if not os.path.exists(to_path):
    os.makedirs(to_path)

for name in ['test.csv','valid.csv']: 
    cmd = 'cp {} {}'.format(from_path+'/'+name, to_path+'/')
    if not os.path.exists(to_path+'/'+name):
        print(cmd)
        os.system(cmd)

cp ../criteo_x1/test.csv ../criteo_x1_0.9/
cp ../criteo_x1/valid.csv ../criteo_x1_0.9/


In [4]:
import pickle
if not os.path.exists(to_path+'/train_backup.pkl'):
    train = pd.read_csv(from_path+'/train.csv')
    with open('./train_backup.pkl','wb') as f:
        pickle.dump(train,f)
else:
    with open('./train_backup.pkl','rb') as f:
        train = pickle.load(f)
train.head(3)

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.05,0.006633,0.05,0.00,0.021594,0.008,0.15,0.04,0.362,...,1528982,1529013,1533925,1536019,1556876,1934144,1934164,1936312,2022803,2024738
1,0,0.10,0.004975,0.44,0.02,0.001594,0.016,0.02,0.04,0.008,...,1528983,1529008,1533925,1536020,1536034,1934144,1934164,1934195,2022803,2022906
2,0,0.10,0.004975,0.01,0.28,0.011984,0.178,0.04,0.04,0.490,...,1528988,1529304,1533924,1536018,1536025,1934145,1934164,1934181,2022801,2022897


In [5]:
negative_index = train[train['label']==0].index
np.random.seed(42)
sampled_negative_index = np.random.choice(a=negative_index, size=int(len(negative_index)*negative_sample_ratio),replace=False)
print("Negative :{}->{}, {}.".format(
    len(negative_index), len(sampled_negative_index), len(sampled_negative_index)/ len(negative_index),
))

Negative :24546957->22092261, 0.8999999877785259.


In [6]:
positive = train[train['label']==1]
sampled_negative = train.iloc[sampled_negative_index]
before_len = len(train)
del train
print("Del train")
after = pd.concat([positive,sampled_negative],axis=0).reset_index(drop=True)

Del train


In [7]:
after.head(3)

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1,0.05,0.011609,0.02,0.00,0.000000,0.000,0.01,0.00,0.000,...,1528982,1529022,1533924,1536018,1536022,1934144,1934163,1934185,2022801,2022897
1,1,0.05,1.000000,0.00,0.04,0.001641,0.004,0.01,0.04,0.004,...,1528984,1529001,1533925,1536019,1783027,1934144,1934163,1939176,2022802,2022902
2,1,0.00,0.082919,0.00,0.00,0.099984,0.076,0.19,0.20,0.286,...,1528982,1529114,1533924,1536018,1536067,1934144,1934163,1934233,2022801,2022897


In [8]:
print("Before {}, After {}, {}.".format(before_len, len(after), len(after)/before_len))

Before 33003326, After 30548630, 0.9256227690506103.


In [9]:
csv_path = to_path+'/train.csv'

# with pd.HDFStore(h5_file_path, mode='w') as store:
#     for column_name, column_data in tqdm(after.items()):
#         key = column_name
#         store[key] = column_data
after.to_csv(csv_path)
print("Saved: {}".format(csv_path))

Saved: ../criteo_x1_0.9/train.csv


In [10]:
"""
0.1
Negative :24546957->2454695, 0.09999997148322702.
Before 33003326, After 10911064, 0.330604982055445.

0.2
Negative :24546957->4909391, 0.19999998370470115.
Before 33003326, After 13365760, 0.4049822130048347.

0.3
Negative :24546957->7364087, 0.29999999592617527.
Before 33003326, After 15820456, 0.4793594439542245.

0.4
Negative :24546957->9818782, 0.3999999674094023.
Before 33003326, After 18275151, 0.5537366446036378.

0.5
Negative :24546957->12273478, 0.49999997963087645.
Before 33003326, After 20729847, 0.6281138755530276.

0.6
Negative :24546957->14728174, 0.5999999918523505
Before 33003326, After 23184543, 0.7024911065024174.

0.7
Negative :24546957->17182869, 0.6999999633355776.
Before 33003326, After 25639238, 0.7768683071518308.

0.8
Negative :24546957->19637565, 0.7999999755570517.
Before 33003326, After 28093934, 0.8512455381012205.

0.9
Negative :24546957->22092261, 0.8999999877785259.
Before 33003326, After 30548630, 0.9256227690506103.
"""

'\n0.1\nNegative :24546957->2454695, 0.09999997148322702.\nBefore 33003326, After 10911064, 0.330604982055445.\n\n0.2\nNegative :24546957->4909391, 0.19999998370470115.\nBefore 33003326, After 13365760, 0.4049822130048347.\n\n0.3\nNegative :24546957->7364087, 0.29999999592617527.\nBefore 33003326, After 15820456, 0.4793594439542245.\n\n0.4\nNegative :24546957->9818782, 0.3999999674094023.\nBefore 33003326, After 18275151, 0.5537366446036378.\n\n0.5\nNegative :24546957->12273478, 0.49999997963087645.\nBefore 33003326, After 20729847, 0.6281138755530276.\n\n0.6\nNegative :24546957->14728174, 0.5999999918523505\nBefore 33003326, After 23184543, 0.7024911065024174.\n\n0.7\nNegative :24546957->17182869, 0.6999999633355776.\nBefore 33003326, After 25639238, 0.7768683071518308.\n\n0.8\nNegative :24546957->19637565, 0.7999999755570517.\nBefore 33003326, After 28093934, 0.8512455381012205.\n\n0.9\nNegative :24546957->22092261, 0.8999999877785259.\n'

: 